In [14]:
import math
import random
import numpy as np

In [15]:
sta_amount = 10
step = 4
p_min = 10
p_max = 250

retrans_counter = [0,1,2,3,4,4,4,4,4,]
sta_power = [p_min + ((p_max-p_min)/(step-1)) * random.randint(0, step-1) for x in retrans_counter]
sta_power

[10.0, 250.0, 90.0, 170.0, 170.0, 250.0, 170.0, 10.0, 10.0]

In [16]:
# sta_amount: Total contend stations number
# RA_RU: RA_RU number reserved for UORA
# Lmax: Retransmit limit
# TXOP: Transaction Opportunity (time length in ms)
# ocw_min: minimum OCW
# ocw_max: maximum OCW
# CR: Capture Ratio (CR=infinite = no capture, CR=0 = always capture)

def simulation(sta_amount, RA_RU, Lmax, TXOP, ocw_min, ocw_max, CR, gamma, p_min, p_max):
    total_power = 0
    counter_delay_slot=0
    success_sta=0
    # setting ocw for each transmission [ocw_min, 1st_retry_ocw...ocw_max]
    ocw = [0 for _ in range(0, Lmax)]
    ocw[0] = ocw_min
    for n in range (1, Lmax):
        temp_ocw = ocw[n-1] * 2 + 1
        if temp_ocw <= ocw_max:
            ocw[n] = temp_ocw
        else:
            ocw[n] = ocw[n-1]
    # Imax: Reserved slot for transmission (total slots in worst case)
    Imax = 0
    for n in range(0, Lmax):
        Imax += math.ceil(ocw[n] / RA_RU)
    # ------------------------------------ STA setting ------------------------------------------
    # set each STA's OBO counter, obo_counter[STA1_obo_counter, STA2_obo_counter...]
    obo_counter = [random.randint(0, ocw_min) for _ in range(0, sta_amount)]
    # set STA's transmit attempt time, retrans_counter[STA1_obo_counter, STA2_obo_counter...]
    retrans_counter = [0 for _ in range(0, sta_amount)]
    # STA's distance from AP (From 0~1)
    sta_distance = [random.uniform(0, 1) for _ in range(0, sta_amount)]
    # record STAs choosen RA-RU, sta_on_R[slot_number][STA_1, STA_26...]
    sta_on_R = [[[] for _ in range(0,RA_RU)] for _ in range(0,Imax)]
    # Accumulated station transmit power usage
    sta_power_usage = [0 for _ in range(0, sta_amount)]
    
    capture_power_ratio = 10 ** (CR / 10)

    # No.i slot    
    for i in range(0, Imax):
        # STA's Transmit power [1P, 2P, 4P, 8P]
        sta_power = [p_min + ((p_max-p_min)/(Lmax-1)) * random.randint(0, Lmax-1) for x in retrans_counter]
        # No.x STA
        for x in range (0,sta_amount):
            # decrease OBO counter by RA-RU number
            obo_counter[x] -= RA_RU
            # if counter <=0 select R
            if (obo_counter[x] <= 0 and retrans_counter[x] < Lmax):
                selected_R_index = random.randint(0, RA_RU-1)
                sta_on_R[i][selected_R_index].append(x)
                
                # record station power usage
                sta_power_usage[x] += sta_power[x]
        # check the balls in R
        for r in range(0, len(sta_on_R[i])):
            # Record power (no power control so always 1)
            for sta_index in sta_on_R[i][r]:
                total_power += sta_power[sta_index]
            # success STA
            if (len(sta_on_R[i][r]) == 1):
                success_sta_index = sta_on_R[i][r][0]
                # mark 99999 = won't trasnmit again
                obo_counter[success_sta_index] = 99999
                # record access delay slot number
                counter_delay_slot += (i+1)
                success_sta += 1
#                 total_power += sta_power_usage[success_sta_index]
            # 2 STA on the same R -> capture process
            elif (len(sta_on_R[i][r]) == 2):
                success_sta_index_1 = sta_on_R[i][r][0]
                success_sta_index_2 = sta_on_R[i][r][1]
                receive_power_1 = sta_power[success_sta_index_1] / (sta_distance[success_sta_index_1] ** gamma)
                receive_power_2 = sta_power[success_sta_index_2] / (sta_distance[success_sta_index_2] ** gamma)
                
                if receive_power_1 >= capture_power_ratio * receive_power_2:
                    # Capture sta 1
                    obo_counter[success_sta_index_1] = 99999
                    counter_delay_slot += (i+1)
                    success_sta += 1
#                     total_power += sta_power_usage[success_sta_index_1]
                    
                    # fail STAs 2
                    retrans_counter[success_sta_index_2] += 1
                    if (retrans_counter[success_sta_index_2] < Lmax):
                        obo_counter[success_sta_index_2] = random.randint(0, ocw[retrans_counter[success_sta_index_2]])
                elif receive_power_2 >= capture_power_ratio * receive_power_1:
                    # Capture sta 2
                    obo_counter[success_sta_index_2] = 99999
                    counter_delay_slot += (i+1)
                    success_sta += 1
#                     total_power += sta_power_usage[success_sta_index_2]
                    
                    # fail STA 1
                    retrans_counter[success_sta_index_1] += 1
                    if (retrans_counter[success_sta_index_1] < Lmax):
                        obo_counter[success_sta_index_1] = random.randint(0, ocw[retrans_counter[success_sta_index_1]])
                else:
                    retrans_counter[success_sta_index_1] += 1
                    if (retrans_counter[success_sta_index_1] < Lmax):
                        obo_counter[success_sta_index_1] = random.randint(0, ocw[retrans_counter[success_sta_index_1]])
                    retrans_counter[success_sta_index_2] += 1
                    if (retrans_counter[success_sta_index_2] < Lmax):
                        obo_counter[success_sta_index_2] = random.randint(0, ocw[retrans_counter[success_sta_index_2]])
                
            else:
                for fail_idx in sta_on_R[i][r]:
                    retrans_counter[fail_idx] += 1
                    if (retrans_counter[fail_idx] < Lmax):
                        obo_counter[fail_idx] = random.randint(0, ocw[retrans_counter[fail_idx]])
    success_probability = success_sta / sta_amount
    access_delay = 0
    if success_sta > 0:
        access_delay = (TXOP * counter_delay_slot) / success_sta
    
    through_put = success_sta / Imax
    utilization = success_sta / (Imax * RA_RU)
    
    average_power = 0
    if success_sta > 0:
        average_power = total_power / success_sta
#     average_power = total_power / sta_amount
    
    return success_probability, access_delay, through_put, utilization, average_power

In [17]:
sample = 1000

# 1, 2, ... 10, 20, 30, ... 100, 200, 300, ... 1000
m_list_simu = np.concatenate([np.arange(1, 11, 1), np.arange(20, 110, 10), np.arange(200, 1100, 100)])
Ps_list_simu = [0 for i in range(28)]
Da_list_simu = [0 for i in range(28)]
T_list_simu = [0 for i in range(28)]
U_list_simu = [0 for i in range(28)]
power_list_simu = [0 for i in range(28)]

# m_list_simu = [10,20,30,40,50,60,70,80,90,100,110,120,130,140,150,160,170,180,190,200]
# Ps_list_simu = [0 for i in range(20)]
# Da_list_simu = [0 for i in range(20)]
# T_list_simu = [0 for i in range(20)]
# U_list_simu = [0 for i in range(20)]
# power_list_simu = [0 for i in range(20)]

# m_list_simu = [(i+1)*10 for i in range(50)]
# Ps_list_simu = [0 for i in range(50)]
# Da_list_simu = [0 for i in range(50)]
# T_list_simu = [0 for i in range(50)]
# U_list_simu = [0 for i in range(50)]

# R = 148
# CR = float('inf')
step = 4
# R_list = [18, 37, 74]
R_list = [148]
CR_list = [3]
# CR_list = [1.5, 3, 6, float('inf')]
for CR in CR_list:
    print(f'---------------------- CR={CR}----------------------')
    for R in R_list:
        for idx, m in enumerate(m_list_simu):
        #     print(idx)
            success_probability_total = 0;
            access_delay_total = 0;
            through_put_total = 0;
            utilization_total = 0;
            no_success_counter = 0;
            power_usage_total = 0;
            for i in range(sample):
                success_probability, access_delay, through_put, utilization, power_usage = simulation(sta_amount=m, RA_RU=R, Lmax=5, TXOP=5.673, ocw_min=7, ocw_max=31, CR=CR, gamma=2, p_min=10, p_max=250)
                success_probability_total += success_probability
                access_delay_total += access_delay
                through_put_total += through_put
                utilization_total += utilization
                power_usage_total += power_usage
                if success_probability == 0:
                    no_success_counter += 1

            Ps = success_probability_total / sample
#             Da = access_delay_total / (sample - no_success_counter)
            Da = access_delay_total / sample
            T = through_put_total / sample
            U = utilization_total / sample
            if (sample - no_success_counter) > 0:
                power = power_usage_total / (sample - no_success_counter)
#             power = power_usage_total / sample


            Ps_list_simu[idx] = Ps
            Da_list_simu[idx] = Da
            T_list_simu[idx] = T
            U_list_simu[idx] = U
            power_list_simu[idx] = power

        print(f'ps_list_simu_{R}_random = {Ps_list_simu}')
        print(f'da_list_simu_{R}_random = {Da_list_simu}')
        print(f't_list_simu_{R}_random = {T_list_simu}')
        print(f'u_list_simu_{R}_random = {U_list_simu}')
        print(f'power_list_simu_{R}_random = {power_list_simu}')
#         print(f'ps_list_simu_{R} = {Ps_list_simu}')
#         print(f'da_list_simu_{R} = {Da_list_simu}')
#         print(f't_list_simu_{R} = {T_list_simu}')
#         print(f'u_list_simu_{R} = {U_list_simu}')
#         print(f'power_list_simu_{R} = {power_list_simu}')
        print("\n")

---------------------- CR=3----------------------
ps_list_simu_148_random = [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.999995, 0.9933066666666719, 0.855787499999999, 0.5398739999999995, 0.28818833333333327, 0.1502528571428568, 0.07757249999999993, 0.04106555555555542, 0.02194499999999989]
da_list_simu_148_random = [5.672999999999856, 5.6900189999998565, 5.724056999999858, 5.728311749999864, 5.768306399999869, 5.774168499999862, 5.801858142857024, 5.82546187499989, 5.853905666666553, 5.851699499999907, 6.123719849999995, 6.330122499999993, 6.566497500000021, 6.766640940000013, 7.013057149999999, 7.240611985714272, 7.478148600000041, 7.718494700000009, 7.9601834099999875, 10.805561227386946, 14.55918318603129, 17.79347460534979, 18.61969885584665, 18.380517790909593, 17.973002848864926, 17.617969053265927, 17.41707780871471, 17.259028659316265]
t_list_simu_148_random = [0.19999999999999718, 0.39999999999999436, 0.6000000000000113, 0.